# SVM Model Evaluation & Validation
## Overview

This project is based on the original work: "Building Predictive Models for Mechanism-of-Action Classification from Phenotypic Assay Data Sets". The aim is to get a basic working idea of what's going on and get a working model. It is NOT intended to exactly replicate what the original work was doing. 

The [original work](http://journals.sagepub.com/doi/abs/10.1177/1087057113505324) built a predictive model for assigning mechanism class to compounds and bioactive agents. The model used 84 features and 309 environmental chemicals. The mechanism classes can used to evaluate the safety and efficacy of compounds and include classes such as inhibitors of mitochondrial and microtubule function, etc.

## Read the data


## Features

----
## Data Exploration

### Calculate Statistics

### Profile plots

### Mechanism box plots

### Feature observations

----
## Develop Model

### Define performance metric

### Does the model capture differences among classes?

----
## Analyzing Model Performance
This section we'll take a look at the model's learning and testing performances on various subsets of training data.  Graphing your model's performance based on varying criteria can be beneficial in the analysis process, such as visualizing behavior that may not have been apparent from the results alone.

### Learning Curves
This graph visalizes the SVM both model's traiing and testing performance with increasing data set size. The shaded region denotes uncertainty of the curve measured in standard deviation.
The model is scored on both the training and testing sets using R2, the coefficient of determination.



-----
## Evaluating Model Performance
We'll construct a model and make a prediction on the compounds in client data sets, using the optimized model.

### Grid Search
 - Grid search is used to find the optimal values of a learning algorithm. It is used to calculate many performace scores of a learning algorithm corresponding to a range of hyper-parameter values and pick the best set of hyper-parameter values. It is a 'grid' in the sense that permutations of hyper-parameter values calculated that can be summarized and displayed in grid or a table.
 - Grid search can be applied to optimize a learning algorithm by running it with a range of hyper-parameters to see which combination of parameters perform best. A set of hyper-parameretes are varied between predefined ranges of values and the model is trained with these values. The optimal set of hyper-parameters are then reported. The model is then tested (evaluated) using a portion of the data not used for training.
 - The hyper parameter of importance when working with SVM is C. To balance margin violations and keeping the separation as wide as possible, SVM algorithm has the hyper-parameter C to control this balance. Reducing C will generalize the model better because it emphasizes the regularization term. Large value of C gives higher weight to the individual feaures while therefore it increases variance. 

----
## Making Predictions
The SVM model has been trained on the given set of data, it can now be used to make predictions on new sets of input compounds. We can use these prediciond to gain information about a compound exhibits featues that make it likely to be in a Mechanism class.
